In [5]:
import pandas as pd
import numpy as np
from utils.data import load_mnist, load_mnist_dataloaders
from utils.neural_network_pytorch import NeuralNetwork, training_loop
import torch.nn as nn
import torch.optim as optim

In [6]:
train_set, test_set = load_mnist()
train_dataloader, test_dataloader = load_mnist_dataloaders(train_set, test_set, batch_size=64)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4310452.98it/s]


Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 234919.81it/s]


Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 1887563.07it/s]


Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 672593.16it/s]

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



In [3]:
activation_function = nn.ReLU()
model = NeuralNetwork(input_size=784, hidden_sizes=[28], output_size=10, activation_function=activation_function)
optimizer = optim.Adam(model.parameters(), lr=1e-6)
loss_function = nn.CrossEntropyLoss()

In [4]:
training_loop(train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              model=model,
              loss_function=loss_function,
              optimizer=optimizer,
              num_epochs=15)

epoch 1/15 | train loss: 2.264930486679077 | test_loss: 2.2636194229125977
epoch 2/15 | train loss: 2.249565601348877 | test_loss: 2.2062926292419434
epoch 3/15 | train loss: 2.1686105728149414 | test_loss: 2.1494784355163574
epoch 4/15 | train loss: 2.142019271850586 | test_loss: 2.0981223583221436
epoch 5/15 | train loss: 2.1266045570373535 | test_loss: 2.0420124530792236
epoch 6/15 | train loss: 2.089233875274658 | test_loss: 1.9834729433059692
epoch 7/15 | train loss: 2.0236141681671143 | test_loss: 1.9269071817398071
epoch 8/15 | train loss: 1.9627858400344849 | test_loss: 1.8673276901245117
epoch 9/15 | train loss: 1.991352915763855 | test_loss: 1.8035719394683838
epoch 10/15 | train loss: 1.9818849563598633 | test_loss: 1.7439292669296265
epoch 11/15 | train loss: 1.7617274522781372 | test_loss: 1.6865408420562744
epoch 12/15 | train loss: 1.7753971815109253 | test_loss: 1.6313859224319458
epoch 13/15 | train loss: 1.6791592836380005 | test_loss: 1.577243447303772
epoch 14/15 | 